<a href="https://colab.research.google.com/github/Naveed2204016/PoliMemeDecode/blob/main/backup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
# =========================
# 1. Imports & Setup
# =========================
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, Concatenate
from keras.models import Model

!pip install -q transformers easyocr

import easyocr
from transformers import TFAutoModel, AutoTokenizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 15.6 MB/s eta 0:00:00


In [3]:

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c poli-meme-decode-cuet-cse-fest

 86% 312M/364M [00:00<00:00, 656MB/s]
100% 364M/364M [00:00<00:00, 637MB/s]


In [5]:

import zipfile
zip_ref=zipfile.ZipFile('/content/poli-meme-decode-cuet-cse-fest.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [6]:
# =========================
# 2. Load Data
# =========================
train_df = pd.read_csv("/content/PoliMemeDecode/Train/Train.csv")
train_dir = "/content/PoliMemeDecode/Train/Image/"
train_df["filepath"] = train_df["Image_name"].apply(lambda x: os.path.join(train_dir, x))
label_map = {"Political": 1, "NonPolitical": 0}
train_df["label"] = train_df["Label"].map(label_map)

paths = train_df["filepath"].values
labels = train_df["label"].values

In [7]:
# =========================
# 2. Load Data
# =========================
train_df = pd.read_csv("/content/PoliMemeDecode/Train/Train.csv")
train_dir = "/content/PoliMemeDecode/Train/Image/"
train_df["filepath"] = train_df["Image_name"].apply(lambda x: os.path.join(train_dir, x))
label_map = {"Political": 1, "NonPolitical": 0}
train_df["label"] = train_df["Label"].map(label_map)

paths = train_df["filepath"].values
labels = train_df["label"].values

In [8]:
# =========================
# 3. Prepare OCR + Text Encoder (multilingual)
# =========================
import torch
import easyocr
from transformers import AutoTokenizer, AutoModel

# OCR for Bangla + English
reader = easyocr.Reader(['bn', 'en'])

# Use PyTorch model instead of TF
TEXT_MODEL_NAME = "bert-base-multilingual-uncased"

tokenizer = AutoTokenizer.from_pretrained(TEXT_MODEL_NAME)
text_encoder = AutoModel.from_pretrained(TEXT_MODEL_NAME)   # <-- FIXED

# Function to extract CLS embedding (returns numpy 1D vector)
def encode_text(text_list):
    enc = tokenizer(text_list, padding='max_length', truncation=True,
                    max_length=128, return_tensors="pt")  # pytorch tensors

    with torch.no_grad():
        outputs = text_encoder(**enc)

    cls_emb = outputs.last_hidden_state[:, 0, :]  # (batch, hidden_size)
    return cls_emb.cpu().numpy()

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

In [9]:
from keras.applications import EfficientNetV2B0

IMG_SIZE = (224, 224, 3)

# Use EfficientNetV2B0 instead of EfficientNetB0
conv_base = EfficientNetV2B0(
    weights="imagenet",
    include_top=False,
    include_preprocessing=True,   # uses Keras layers, NOT TF ops
    input_shape=IMG_SIZE
)
conv_base.trainable = False

img_input = Input(shape=IMG_SIZE, name="image_input")
txt_input = Input(shape=(768,), name="text_input")

# This WILL WORK — no rescaling error
x_img = conv_base(img_input)
x_img = GlobalAveragePooling2D()(x_img)

combined = Concatenate()([x_img, txt_input])
x = Dense(256, activation="relu")(combined)
x = Dropout(0.4)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.4)(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=[img_input, txt_input], outputs=output)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

model.summary()

24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ efficientnetv2-b0   │ (None, 7, 7,      │  5,919,312 │ image_input[0][0] │
│ (Functional)        │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1280)      │          0 │ efficientnetv2-b… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ text_input          │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 2048)      │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ text_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │    524,544 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     32,896 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │        129 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,476,881 (24.71 MB)

 Trainable params: 557,569 (2.13 MB)

 Non-trainable params: 5,919,312 (22.58 MB)

In [10]:
import os
import cv2
import numpy as np
from tqdm import tqdm

train_dir ="/content/PoliMemeDecode/Train/Image/"
test_dir ="/content/PoliMemeDecode/Test/Image/"

# Load CSVs
train_df = pd.read_csv("/content/PoliMemeDecode/Train/Train.csv")   # Image_name, Label
test_df  = pd.read_csv("/content/PoliMemeDecode/Test/Test.csv")    # Image_name only

# Add full paths
train_df["filepath"] = train_df["Image_name"].apply(lambda x: os.path.join(train_dir, x))
test_df["filepath"] = test_df["Image_name"].apply(lambda x: os.path.join(test_dir, x))

In [11]:
def extract_ocr_text(img_path):
    try:
        result = reader.readtext(img_path, detail=0)
        joined = " ".join(result)
        return joined if joined.strip() != "" else "no_text"
    except:
        return "no_text"


In [12]:
train_texts = []
for fp in tqdm(train_df["filepath"], desc="OCR Train"):
    train_texts.append(extract_ocr_text(fp))

train_df["ocr_text"] = train_texts


OCR Train: 100%|██████████| 2860/2860 [23:16<00:00,  2.05it/s]


In [13]:
def encode_text_in_batches(text_list, batch_size=32):
    import numpy as np
    all_embeddings = []

    for i in range(0, len(text_list), batch_size):
        batch = text_list[i : i + batch_size]

        enc = tokenizer(
            batch,
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors="pt"
        )

        with torch.no_grad():
            outputs = text_encoder(**enc)

        cls_emb = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        all_embeddings.append(cls_emb)

    return np.vstack(all_embeddings)

# Now run this instead of encode_text(...)
train_text_embeddings = encode_text_in_batches(
    train_df["ocr_text"].tolist(),
    batch_size=16   # if RAM low, use 8
)


In [21]:
train_df.head()


,Image_name,Label,filepath,ocr_text
0,train0001.jpg,Political,/content/PoliMemeDecode/Train/Image/train0001.jpg,হরেক রকম চোর দিয়া বানাইছো মেলা পিসি তোমার দরব...
1,train0002.jpg,NonPolitical,/content/PoliMemeDecode/Train/Image/train0002.jpg,বর যাত্রী যাওয়ার টাইমে যখব গাড়িতে জায়গা কম পড়...
2,train0003.jpg,NonPolitical,/content/PoliMemeDecode/Train/Image/train0003.jpg,তুমিকেখা নোবলল বুকে অনেক ব্যখাহেয় TARLBAU; ঠিক...
3,train0004.jpg,NonPolitical,/content/PoliMemeDecode/Train/Image/train0004.jpg,যখন ভল্লুক এসে পড়ে এবং আপন এবং আপনার বেস্টফ়র...
4,train0005.jpg,NonPolitical,/content/PoliMemeDecode/Train/Image/train0005.jpg,বন্ধুযখন তার গফকেকাজিন বলে পরিচয় দেয় [41#4 {...


In [22]:
from sklearn.model_selection import train_test_split

paths_train, paths_val, labels_train, labels_val = train_test_split(
    train_df["filepath"].values,
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)


In [23]:
IMG_SIZE = (224, 224)

def load_image(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    return img, label

train_ds = tf.data.Dataset.from_tensor_slices((paths_train, labels_train))
train_ds = train_ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((paths_val, labels_val))
val_ds = val_ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)


In [24]:
# Extract OCR texts for validation set
val_df = train_df[train_df["filepath"].isin(paths_val)]

val_text_embeddings = encode_text_in_batches(
    val_df["ocr_text"].tolist(),
    batch_size=16
)


In [25]:
train_text_embeddings_tf = tf.convert_to_tensor(train_text_embeddings, dtype=tf.float32)
val_text_embeddings_tf   = tf.convert_to_tensor(val_text_embeddings, dtype=tf.float32)


In [30]:
def add_text_embeddings(data, idx):
    image, label = data        # unpack
    text_emb = train_text_embeddings_tf[idx]
    return {"image_input": image, "text_input": text_emb}, label


def add_text_embeddings_val(data, idx):
    image, label = data
    text_emb = val_text_embeddings_tf[idx]
    return {"image_input": image, "text_input": text_emb}, label


In [31]:
train_idx_ds = tf.data.Dataset.range(len(train_text_embeddings))
val_idx_ds   = tf.data.Dataset.range(len(val_text_embeddings))

# Train dataset
train_final_ds = tf.data.Dataset.zip((train_ds, train_idx_ds))
train_final_ds = train_final_ds.map(add_text_embeddings, num_parallel_calls=tf.data.AUTOTUNE)
train_final_ds = train_final_ds.batch(32).prefetch(tf.data.AUTOTUNE)

# Val dataset
val_final_ds = tf.data.Dataset.zip((val_ds, val_idx_ds))
val_final_ds = val_final_ds.map(add_text_embeddings_val, num_parallel_calls=tf.data.AUTOTUNE)
val_final_ds = val_final_ds.batch(32).prefetch(tf.data.AUTOTUNE)


In [32]:
history = model.fit(
    train_final_ds,
    validation_data=val_final_ds,
    epochs=5
)


Epoch 1/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 224s 2s/step - accuracy: 0.6785 - loss: 0.6247 - val_accuracy: 0.7745 - val_loss: 0.4856
Epoch 2/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 47s 659ms/step - accuracy: 0.7753 - loss: 0.4724 - val_accuracy: 0.7885 - val_loss: 0.4613
Epoch 3/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 83s 675ms/step - accuracy: 0.8053 - loss: 0.4149 - val_accuracy: 0.7955 - val_loss: 0.4468
Epoch 4/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 81s 657ms/step - accuracy: 0.8365 - loss: 0.3867 - val_accuracy: 0.7955 - val_loss: 0.4525
Epoch 5/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 84s 689ms/step - accuracy: 0.8494 - loss: 0.3432 - val_accuracy: 0.8164 - val_loss: 0.4489


In [33]:
test_texts = []
for fp in tqdm(test_df["filepath"], desc="OCR Test"):
    test_texts.append(extract_ocr_text(fp))

test_df["ocr_text"] = test_texts


OCR Test: 100%|██████████| 330/330 [05:36<00:00,  1.02s/it]


In [34]:
test_text_embeddings = encode_text_in_batches(
    test_df["ocr_text"].tolist(),
    batch_size=16
)


In [35]:
def load_test_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    return img

test_img_ds = tf.data.Dataset.from_tensor_slices(test_df["filepath"].values)
test_img_ds = test_img_ds.map(load_test_image, num_parallel_calls=tf.data.AUTOTUNE)

test_text_embeddings_tf = tf.convert_to_tensor(test_text_embeddings, dtype=tf.float32)

test_idx_ds = tf.data.Dataset.range(len(test_text_embeddings))


In [37]:
def add_test_text_embeddings(image, idx):
    text_emb = test_text_embeddings_tf[idx]
    return {"image_input": image, "text_input": text_emb}


In [38]:
# Index dataset for test
test_idx_ds = tf.data.Dataset.range(len(test_text_embeddings))

# Zip: (image) + (index)
test_final_ds = tf.data.Dataset.zip((test_img_ds, test_idx_ds))

# Map with two-argument function
test_final_ds = test_final_ds.map(
    add_test_text_embeddings,
    num_parallel_calls=tf.data.AUTOTUNE
)

# Batch + Prefetch
test_final_ds = test_final_ds.batch(32).prefetch(tf.data.AUTOTUNE)


In [39]:
preds = model.predict(test_final_ds)
preds = (preds > 0.5).astype(int).reshape(-1)


11/11 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step


In [40]:
# Convert predictions (0/1) → string labels
inv_label_map = {1: "Political", 0: "NonPolitical"}

pred_labels = [inv_label_map[p] for p in preds]

# Build submission dataframe
submission = pd.DataFrame({
    "Image_name": test_df["Image_name"],
    "Label": pred_labels
})

submission.to_csv("submission3.csv", index=False)
#submission.head()
